In [1]:
import csv
import sqlite3
import functions
import pandas as pd
import requests as rq
from config import api_key
# pip install geopy
from geopy.geocoders import Nominatim
import pickle

In [2]:
# df = pd.read_csv('Resources/airlines_reviews.csv')
# df.head(2)

In [3]:
# conn = sqlite3.connect('airline_passenger_satisfaction.sqlite')
# df.to_sql('airlines_reviews', conn, if_exists='replace')
# db = conn.cursor()
# db.execute('SELECT * FROM airlines_reviews').fetchall()

In [5]:
# Loading model to compare the results ### model = pickle.load(open('model.pkl','rb'))
df = pickle.load(open('df.pkl','rb'))
df

,Title,Airline,Reviews,Month Flown,Route,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,...,Destination_City,Via_City,Class_Business Class,Class_Economy Class,Class_First Class,Class_Premium Economy,Type of Traveller_Business,Type of Traveller_Couple Leisure,Type of Traveller_Family Leisure,Type of Traveller_Solo Leisure
0,Flight was amazing,Singapore Airlines,Flight was amazing. The crew onboard this fl...,12,Jakarta to Singapore,4,4,4,4,4,...,Singapore,None,True,False,False,False,False,False,False,True
1,seats on this aircraft are dreadful,Singapore Airlines,Booking an emergency exit seat still meant h...,2,Phuket to Singapore,5,3,4,4,1,...,Singapore,None,False,True,False,False,False,False,False,True
2,Food was plentiful and tasty,Singapore Airlines,Excellent performance on all fronts. I would...,2,Siem Reap to Singapore,1,5,2,1,5,...,Singapore,None,False,True,False,False,False,False,True,False
3,“how much food was available,Singapore Airlines,Pretty comfortable flight considering I was f...,2,Singapore to London Heathrow,5,5,5,5,5,...,London Heathrow,None,False,True,False,False,False,False,False,True
4,“service was consistently good”,Singapore Airlines,The service was consistently good from start ...,2,Singapore to Phnom Penh,5,5,5,5,5,...,Phnom Penh,None,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,an uneventful flight,Korean Air,"KE124, Brisbane to Incheon (A330) and KE867,...",6,BNE to ULN via ICN,5,4,5,3,4,...,Ulaanbaatar,"Incheon, Incheon Int'l Airport",False,True,False,False,True,False,False,False
8096,Korean Air always impresses,Korean Air,Our recent flight was our fourth trip to the...,6,SYD to LHR via ICN,3,5,5,4,5,...,Heathrow,"Incheon, Incheon Int'l Airport",False,True,False,False,False,True,False,False
8097,didn’t offer anything,Korean Air,I flew Korean Air from Bali to Seoul in Pres...,4,DPS to ICN,4,5,5,5,1,...,"Incheon, Incheon Int'l Airport",None,True,False,False,False,True,False,False,False
8098,appreciated the service onboard,Korean Air,Seoul to Paris with Korean Air. I am traveli...,4,ICN to CDG,5,1,3,4,5,...,Paris cdg,None,True,False,False,False,True,False,False,False


In [50]:
unique_codes = df['Origin_Code'].unique()
len(unique_codes)

365

In [51]:
unique_codes_Dest = df['Destination_Code'].unique()
len(unique_codes_Dest)

362

In [53]:
unique_codes = set(unique_codes)
unique_codes_Dest = set(unique_codes_Dest)
unique_codes = unique_codes.union(unique_codes_Dest)

In [54]:
len(unique_codes)

430

In [57]:
custom_user_agent = "MyCustomUserAgent"
geolocator = Nominatim(user_agent="custom_user_agent")

In [58]:
import time

def get_lat_long(airport_code):
    geolocator = Nominatim(user_agent="custom_user_agent")
    location = geolocator.geocode(f"{airport_code} airport")
    print(airport_code, location)
    time.sleep(1)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

airport_pos = {code:get_lat_long(code) for code in unique_codes}
airport_pos

EWR Newark Liberty International Airport, US 1-9 Local, Newark, Essex County, New Jersey, 07114, United States
UPG Bandar Udara Internasional Sultan Hasanuddin, Jalan BTN Laikang Indah, Bakung, Maros, Makassar, Sulawesi Selatan, Sulawesi, 90242, Indonesia
VIE Vienna Airport, Towerstraße, Schwechat, Bezirk Bruck an der Leitha, Niederösterreich, 1300, Österreich
TNR Tanguturu, APSRTC Bus Stand Road, Tanguturu, Tangutur, Prakasam, Andhra Pradesh, 523274, India
KUF Курумоч, «Урал», Красноглинский район, Самара, сельское поселение Светлое Поле, городской округ Самара, Самарская область, Приволжский федеральный округ, 446397, Россия
ZAG Zračna luka Franjo Tuđman, Ulica Rudolfa Fizira, Gradska četvrt Pleso, Velika Gorica, Grad Velika Gorica, Zagrebačka županija, 10410, Hrvatska
OSL Oslo lufthavn, Gardermoen, Gardermovegen, Kneppefeltet, Søndre Børke, Nannestad, Viken, 2060, Norge
BOG Aeropuerto Internacional El Dorado, Carrera 116C Bis A, La Faena, UPZs de Bogotá, Localidad Engativá, Bogotá, 

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=DAL&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [45]:
def get_lat_long(airport_code):
    geolocator = Nominatim(user_agent="custom_user_agent")
    location = geolocator.geocode(airport_code)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Apply the function to create new columns for Origin_Latitude, Origin_Longitude, Destination_Latitude, Destination_Longitude for the first 10 rows
df_subset = df.head(10)
df_subset['Origin_Latitude'], df_subset['Origin_Longitude'] = zip(*df_subset['Origin_Code'].apply(get_lat_long))
df_subset['Destination_Latitude'], df_subset['Destination_Longitude'] = zip(*df_subset['Destination_Code'].apply(get_lat_long))

# Display the updated DataFrame for the first 10 rows
print(df_subset)

              Airline           Class          Destination  Food & Beverages  \
0  Singapore Airlines  Business Class            Singapore                 4   
1  Singapore Airlines   Economy Class            Singapore                 4   
2  Singapore Airlines   Economy Class            Singapore                 2   
3  Singapore Airlines   Economy Class      London Heathrow                 5   
4  Singapore Airlines   Economy Class           Phnom Penh                 5   
5  Singapore Airlines   Economy Class               Cairns                 5   
6  Singapore Airlines   Economy Class            Frankfurt                 1   
7  Singapore Airlines   Economy Class  Bandar Seri Begawan                 3   
8  Singapore Airlines   Economy Class              Jakarta                 5   
9  Singapore Airlines   Economy Class            Singapore                 4   

   Inflight Entertainment  Month Flown     Origin  Overall Rating  Polarity  \
0                       4           12  

In [47]:
df.head()

,Airline,Class,Destination,Food & Beverages,Inflight Entertainment,Month Flown,Origin,Overall Rating,Polarity,Recommended,...,Via_2,Year Flown,Origin_Code,Destination_Code,Via_Code,Via_2_Code,Origin_City,Destination_City,Via_City,Via_2_City
0,Singapore Airlines,Business Class,Singapore,4,4,12,Jakarta,9,0.384667,yes,...,NaN,2023,CGK,SIN,None,NaN,Jakarta,Singapore,None,NaN
1,Singapore Airlines,Economy Class,Singapore,4,4,2,Phuket,3,-0.212500,no,...,NaN,2024,HKT,SIN,None,NaN,Phuket,Singapore,None,NaN
2,Singapore Airlines,Economy Class,Singapore,2,1,2,Siem Reap,10,0.500000,yes,...,NaN,2024,SAI,SIN,None,NaN,Siem Reap,Singapore,None,NaN
3,Singapore Airlines,Economy Class,London Heathrow,5,5,2,Singapore,10,0.325496,yes,...,NaN,2024,SIN,LHR,None,NaN,Singapore,London Heathrow,None,NaN
4,Singapore Airlines,Economy Class,Phnom Penh,5,5,2,Singapore,10,0.412500,yes,...,NaN,2024,SIN,PNH,None,NaN,Singapore,Phnom Penh,None,NaN


In [49]:
len(unique_Origin)

654

In [ ]:
print(df['Origin_Code'].dtype)
print(df['Destination_Code'].dtype)
print(df['Via_Code'].dtype)
print(df['Via_2_Code'].dtype)

In [ ]:
df['Origin_Code'] = df['Origin_Code'].astype('category')
df['Destination_Code'] = df['Destination_Code'].astype('category')
df['Via_Code'] = df['Via_Code'].astype('category')
df['Via_2_Code'] = df['Via_2_Code'].astype('category')

In [ ]:
print(df['Origin_Code'].dtype)
print(df['Destination_Code'].dtype)
print(df['Via_Code'].dtype)
print(df['Via_2_Code'].dtype)

consider optimizing your code further by looking into other ways to improve the efficiency of your operations, such as using vectorized operations, avoiding unnecessary iterations, and utilizing appropriate indexing techniques

Avoid unnecessary iterations
1. **Use Vectorized Operations**: Instead of iterating over rows, try to perform operations on entire columns or subsets of data using vectorized operations provided by libraries like NumPy and Pandas. These operations are optimized for performance and can significantly speed up your code.

2. **Avoid Iterating Over DataFrame Rows**: If possible, avoid iterating over rows using methods like `iterrows()` or `apply()`. These methods can be slow, especially on large datasets. Instead, try to find ways to apply operations to entire columns or use built-in Pandas functions that operate on the entire DataFrame.

3. **Use GroupBy**: If you need to perform operations on groups of data, consider using the `groupby()` function in Pandas. This allows you to group your data based on certain criteria and apply functions to each group without explicit iteration.

4. **Merge and Join Operations**: Instead of iterating over two DataFrames to perform operations, consider using merge and join operations provided by Pandas. These operations are optimized and can be more efficient than iterating over rows.

5. **List Comprehensions and Built-in Functions**: When possible, use list comprehensions or built-in functions to perform operations on lists or arrays. These are typically faster than explicit loops and can make your code more concise and readable.

6. **Utilize NumPy Functions**: NumPy provides a wide range of vectorized functions that can operate on arrays efficiently. Wherever possible, try to use NumPy functions instead of writing custom Python loops.

By following these tips and leveraging the capabilities of libraries like Pandas and NumPy, you can avoid unnecessary iterations and improve the performance of your code

In [ ]:
# Convert the column back to its original data type
df['Origin_Code'] = df['Origin_Code'].astype('original_data_type')
df['Destination_Code'] = df['Destination_Code'].astype('original_data_type')
df['Via_Code'] = df['Via_Code'].astype('original_data_type')
df['Via_2_Code'] = df['Via_2_Code'].astype('original_data_type')
print(df['Origin_Code'].dtype)

https://www.developer.aero/api-catalog/flight-duration-overview

In [17]:
####### DO NOT RUN !!!!!!!!!!
# unique_via = df['Via'].unique()
# airports_via = airport_codes['City'].tolist() + airport_codes['Code'].tolist()
# via_values_not_found = [via for via in unique_via if via not in airports_via]

# print("Values from the 'Via' column not found in airports_utf.csv 'City' or 'Code' columns:")
# print(via_values_not_found)

# Values from the 'Via' column not found in airports_utf.csv 'City' or 'Code' columns:
# [None, 'DOA', 'Lagoa', 'DXB/MLE', 'Минск', 'Panama city', '', 'paris', 'Italy', 'Lyons', 'Hongkong', 'Hong kong', 'Degeau', 'INC']

Values from the 'Via' column not found in airports_utf.csv 'City' or 'Code' columns:
[None, 'DOA', 'Lagoa', 'DXB/MLE', 'Минск', 'Panama city', '', 'paris', 'Italy', 'Lyons', 'Hongkong', 'Hong kong', 'Degeau', 'INC']


In [ ]:
### lookup failures, only return cities not found, if return series call unique.apply - return nothing if you can't find it

### get lats/long for origin-destination-via

### https://www.developer.aero/api-catalog/flight-duration-overview

In [ ]:
# keep routes & split into Origin & Destination, find coord for the locations
# Target "Recommended"
# maybe user can provide the following and 
# Type of Traveller', 'Route', 'Class', 'Seat Comfort',
# 'Staff Service', 'Food & Beverages', 'Inflight Entertainment',
# 'Value For Money